In [1]:
import re
from collections import Counter
error_flag=False
def lines_words(file):
    try:
        updated_lines = []
        change_words={}
        for line in file:
            if(not line.startswith("//") and line!="\n"  and not line.startswith('#') and not line.startswith('using')):
                updated_lines.append(line.lower().strip())
            try:
                if(line.startswith("#define")):
                    words = line.split(' ')
                    #print(words)
                    replace = words[1]
                    string = ""
                    for i in range(2,len(words)):
                        string += words[i]
                        string += " "
                    string = string[:-2]
                    #print(string)
                    change_words[replace] = string
            except:
                pass
        code=""
        for i in range(len(updated_lines)):
            code += updated_lines[i]

        code = code.translate({ord("}"):"}; ",ord("("):" ( ",ord("{"):" { ;"})
        for key,value in change_words.items():
            code=code.replace(key,value)

        lines_code=code.split(';')
        return lines_code
    except:
        error_flag=True
        return []

In [2]:
def hashmap_function(lines_code,hashmap):
    try:
        pattern ="\((.*?)\)"
        for i in range(len(lines_code)): 
            if ('{'in lines_code[i] and '(' in lines_code[i] and ')' in lines_code[i] and not('for ' in lines_code[i]) 
                and not('while ' in lines_code[i]) and not('main' in lines_code[i]) and not('if ' in lines_code[i])):

                substring = re.search(pattern, lines_code[i]).group(1)
                #print(substring)
                count_parameter=[]
                type_parameter=[]
                parameters = (substring.split(','))
                #print(parameters)
                parameter_type = [i.strip().rsplit(' ', 1)[0] for i in parameters]
                #print(parameter_type)
                count_parameter_all = dict(Counter(parameter_type))
                for key,value in count_parameter_all.items():
                    if key != "":
                        count_parameter.append(value)
                        type_parameter.append(key)

                #print(lines_code[i])
                words = lines_code[i].strip().split()
                if(words[0]=='static' or words[0]=='inline'):
                    words[0]=''
                return_type=""
                #print(words)
                for j in range(len(words)):
                    if(words[j]=='('):
                        for k in range(j-1):
                            return_type +=words[k]
                            return_type +=" "
                        break
                #print(count_parameter)
                function_signature=(return_type.strip(),tuple(count_parameter),tuple(type_parameter))
                print(function_signature)
                hash_value=hash(function_signature)
                if hash_value not in hashmap:
                    hashmap[hash_value]=1
                else:
                    hashmap[hash_value]+=1
    except:
        error_flag=True
        pass

In [3]:
def common_percentage(hashmap1,hashmap2):
    common=[min(hashmap1[i],hashmap2[i]) for i in hashmap1 if i in hashmap2]
    
    try:
        val = (2*sum(common)/(sum(hashmap1.values())+sum(hashmap2.values())))*100
        if(not error_flag):
            return val
        elif (val!=0):
            return val
        else:
            return 100
    except:
        return 100

In [4]:
hashmap1={}
file = open(r"A.cpp",'r')
lines_code=lines_words(file)
hashmap_function(lines_code,hashmap1)
print(hashmap1)

('temake_pairlate <typename t> std::ostream&', (1, 1), ('std::ostream&', 'const vector<t>&'))
('temake_pairlate <typename t> std::istream&', (1, 1), ('std::istream&', 'vector<t>&'))
('temake_pairlate <typename t,typename u> std::ostream&', (1, 1, 1), ('std::ostream&', 'const', 'u>&'))
('temake_pairlate <typename t,typename u> std::istream&', (1, 1, 1), ('std::istream&', 'pair<t', 'u>&'))
('temake_pairlate <typename t> void', (1, 1), ('vector<t>&', 'const t'))
('temake_pairlate <typename t> void', (1, 1), ('vector<t>&', 'const t'))
('temake_pairlate <typename t> void', (1,), ('vector<t>&',))
('temake_pairlate <typename t> void', (1, 1), ('vector<t>&', 't'))
('temake_pairlate <typename t> void', (1,), ('vector<t>&',))
('temake_pairlate <typename t> void', (1, 1), ('vector<t>&', 't'))
('temake_pairlate <typename t> void', (1, 1), ('vector<t>&', 'const t'))
('temake_pairlate <typename t> void', (1, 1), ('vector<t>&', 'const t'))
('temake_pairlate <typename t,typename u> void', (1, 1, 1), (

In [5]:
hashmap2={}
file = open(r"B.cpp",'r')
lines_code=lines_words(file)
hashmap_function(lines_code,hashmap2)
print(hashmap2)

('', (3, 1), ('std::string', 'int'))
('std::string', (), ())
('std::string', (), ())
('std::string', (), ())
('int', (), ())
('korisnik', (), ())
('void', (1,), ('korisnik',))
('void', (), ())
('bool', (), ())
('', (1,), ('const biblioteka',))
('biblioteka &biblioteka::operator', (1,), ('biblioteka',))
('void', (1, 4), ('int', 'std::string'))
('void', (2, 3), ('int', 'std::string'))
('korisnik', (1,), ('int',))
('const korisnik', (1,), ('int',))
('knjiga', (1,), ('int',))
('const knjiga', (1,), ('int',))
('void', (), ())
('void', (), ())
('void', (2,), ('int',))
('void', (1,), ('int',))
('void', (1,), ('int',))
('try tijelo', (1,), ('izuzetak',))
{-2119768783489209842: 1, -1328274332808593961: 3, -4940995989761889662: 1, -2700487225133768042: 1, 2787761943525592927: 1, 7537188748328037497: 3, 1737325919675198485: 1, -5805233901883222964: 1, -6171969095265571245: 1, 6079725478919717613: 1, -5887702663859038601: 1, -6289211416506410308: 1, 2344651983233139796: 1, 7777042289331044213: 1, 

In [6]:
percentage_same=common_percentage(hashmap1,hashmap2)
percentage_same

12.903225806451612